> We have already established that using annotated HeLa m6A sites, we can observe changes in genes with m6A sites (HL-60) cells. In order to confirm this m6A sites, we performed MeRIP-seq in treated and untreated cells, and did observe a general increase in m6A levels upon treatments for a large number of annotated sites. Here, our goal is to indpendently analyze the MeRIP data without relying on HeLa annotations and use it to define a **treatment-induced hyper-methylation sites**. We will then assess the location and behaviour of these targets across the other datasets generated in this study.

## Test enrichment of treatment-induced hyper/hypo-methylation sites

### Goal
Here, I aim to identify the genes that are hyper or hypo methylated as genesets, and see if they have enriched accross all datasets; the input table is list of genes with control vs. treated fold change of RNA expression, RNA stability and translational efficiency. 
### Steps 
1. Prepare inputs  
    - Filtering genes with $\Delta$methylation >= 1 as hyper-methylation sites (P-Value < 0.01)
    - Filtering genes with $\Delta$methylation <= -1 as hypo-methylation sites (P-Value < 0.01)
2. Run `run_mi_gene_list.pl` command 


In [6]:
import pandas as pd 
import numpy as np
    
def write_gene_file(df,file_name):
    
    df = pd.DataFrame({'ensembl':[ens[:-3] for ens in df.ensembl.tolist()]}).drop_duplicates('ensembl')
    df.to_csv(file_name,sep='\t',index=None,header=None)
    

hyper, hypo = two_sided_mtyl()
write_gene_file(hyper,'mtyl-enrichment/hyper_mtyl.txt')
write_gene_file(hypo,'mtyl-enrichment/hypo_mtyl.txt')

2. Using a [TEISER](https://github.com/goodarzilab/TEISER) script to do enrichment test 



In [11]:
%%bash

declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Experiments=(
# Ribo-seq
'../Ribo-seq/hl60_delta_te.txt'

## HL-60 RNA-seq 
# RNA experssion 
'../RNA-seq/hl60-exp/6h_delta_exp.txt' 
'../RNA-seq/hl60-exp/72h_delta_exp.txt' 
'../RNA-seq/hl60-exp/120h_delta_exp.txt' 
# RNA stability  
'../RNA-seq/hl60-stbl/120h_delta_stbl.txt'  
'../RNA-seq/hl60-stbl/6h_delta_stbl.txt'

## 5 other AML cell lines RNA-seq
# RNA experssion 
'../RNA-seq/other-exp/kg1_delta_exp.txt' 
'../RNA-seq/other-exp/molm14_delta_exp.txt'
'../RNA-seq/other-exp/ociaml2_delta_exp.txt' 
'../RNA-seq/other-exp/ociaml3_delta_exp.txt'
'../RNA-seq/other-exp/thp1_delta_exp.txt'
# RNA stability  
'../RNA-seq/other-stbl/kg1_delta_stbl.txt' 
'../RNA-seq/other-stbl/molm14_delta_stbl.txt' 
'../RNA-seq/other-stbl/ociaml2_delta_stbl.txt' 
'../RNA-seq/other-stbl/ociaml3_delta_stbl.txt'
'../RNA-seq/other-stbl/thp1_delta_stbl.txt'
)

for exp in "${Experiments[@]}"; do
    for geneset in "${Genesets[@]}"; do
    
        echo $exp $geneset
        base=`basename $exp`
        base=${base/.txt/}
        
        # get intersect 
        awk 'NR==FNR{A[$1];next}$1 in A' $exp mtyl-enrichment/${geneset}.txt > mtyl-enrichment/${geneset}_${base}.txt
        
        perl $TEISERDIR/run_mi_gene_list.pl \
            --expfile=$exp \
            --genefile=mtyl-enrichment/${geneset}_${base}.txt \
            --exptype=continuous \
            --ebins=7 \
            --species=human \
            --doremovedups=0 \
            --doremoveextra=0 &> mtyl-enrichment/${geneset}_${base}.log
#         # remove results from previous run 
#         rm -fr mtyl-enrichment/${geneset}_${base}_GENESET
        
        rm mtyl-enrichment/${geneset}_${base}.txt
        mv ${exp}_GENESET mtyl-enrichment/${geneset}_${base}_GENESET
        
        echo 'done!'
        
    done 

done

../Ribo-seq/hl60_delta_te.txt hyper_mtyl
done!
../Ribo-seq/hl60_delta_te.txt hypo_mtyl
done!
../RNA-seq/hl60-exp/6h_delta_exp.txt hyper_mtyl
done!
../RNA-seq/hl60-exp/6h_delta_exp.txt hypo_mtyl
done!
../RNA-seq/hl60-exp/72h_delta_exp.txt hyper_mtyl
done!
../RNA-seq/hl60-exp/72h_delta_exp.txt hypo_mtyl
done!
../RNA-seq/hl60-exp/120h_delta_exp.txt hyper_mtyl
done!
../RNA-seq/hl60-exp/120h_delta_exp.txt hypo_mtyl
done!
../RNA-seq/hl60-stbl/120h_delta_stbl.txt hyper_mtyl
done!
../RNA-seq/hl60-stbl/120h_delta_stbl.txt hypo_mtyl
done!
../RNA-seq/hl60-stbl/6h_delta_stbl.txt hyper_mtyl
done!
../RNA-seq/hl60-stbl/6h_delta_stbl.txt hypo_mtyl
done!
../RNA-seq/other-exp/kg1_delta_exp.txt hyper_mtyl
done!
../RNA-seq/other-exp/kg1_delta_exp.txt hypo_mtyl
done!
../RNA-seq/other-exp/molm14_delta_exp.txt hyper_mtyl
done!
../RNA-seq/other-exp/molm14_delta_exp.txt hypo_mtyl
done!
../RNA-seq/other-exp/ociaml2_delta_exp.txt hyper_mtyl
done!
../RNA-seq/other-exp/ociaml2_delta_exp.txt hypo_mtyl
done!
../RNA-

3. Redraw heatmaps using `--min=-3 --max=3` thresholds for those plots which have smaller range of signals:

In [14]:
%%bash 
declare -a Genesets=('hyper_mtyl' 'hypo_mtyl')
declare -a Experiments=(
'6h_delta_stbl' '120h_delta_stbl' 
'kg1_delta_stbl' 'ociaml2_delta_stbl' 'molm14_delta_stbl' 
'ociaml3_delta_stbl' 'thp1_delta_stbl'
'hl60_delta_te'
)
for exp in "${Experiments[@]}"; do
    for geneset in "${Genesets[@]}"; do
        echo $exp $geneset    
        perl $TEISERDIR/Scripts/teiser_draw_matrix.pl \
        --pvmatrixfile=mtyl-enrichment/${geneset}_${exp}_GENESET/${exp}.txt.matrix \
        --summaryfile=mtyl-enrichment/${geneset}_${exp}_GENESET/${exp}.txt.summary \
        --expfile=mtyl-enrichment/${geneset}_${exp}_GENESET/${exp}.txt \
        --quantized=0 \
        --colmap=$TEISERDIR/Scripts/HEATMAPS/cmap_1.txt \
        --order=0 \
        --min=-3 --max=3 \
        --cluster=5 &>> mtyl-enrichment/${geneset}_${exp}.log
        
        echo 'done!'
        
    done 
done

6h_delta_stbl hyper_mtyl
done!
6h_delta_stbl hypo_mtyl
done!
120h_delta_stbl hyper_mtyl
done!
120h_delta_stbl hypo_mtyl
done!
kg1_delta_stbl hyper_mtyl
done!
kg1_delta_stbl hypo_mtyl
done!
ociaml2_delta_stbl hyper_mtyl
done!
ociaml2_delta_stbl hypo_mtyl
done!
molm14_delta_stbl hyper_mtyl
done!
molm14_delta_stbl hypo_mtyl
done!
ociaml3_delta_stbl hyper_mtyl
done!
ociaml3_delta_stbl hypo_mtyl
done!
thp1_delta_stbl hyper_mtyl
done!
thp1_delta_stbl hypo_mtyl
done!
hl60_delta_te hyper_mtyl
done!
hl60_delta_te hypo_mtyl
done!


4. Make `png` figures:

In [17]:
mkdir -p mtyl-enrichment/plots/

In [19]:
%%bash 
for pdf in mtyl-enrichment/*_GENESET/*.txt.summary.pdf; do 
    png=${pdf/.pdf/.png}
    di=`dirname $pdf`
    out=`basename $di`
    outpng=${out/_GENESET/.png}
    outpdf=${out/_GENESET/.pdf}
    
    bash /rumi/shams/abe/GitHub/Abe/my_scripts/pdf2png.sh $pdf
    cp $pdf mtyl-enrichment/$outpdf
    mv $png mtyl-enrichment/$outpng
done 

mtyl_enrichment/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_120h_delta_exp_GENESET/120h_delta_exp.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_120h_delta_stbl_GENESET/120h_delta_stbl.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_6h_delta_exp_GENESET/6h_delta_exp.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_6h_delta_exp_GENESET/6h_delta_exp.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_6h_delta_stbl_GENESET/6h_delta_stbl.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_72h_delta_exp_GENESET/72h_delta_exp.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_72h_delta_exp_GENESET/72h_delta_exp.txt.summary.png
done!
mtyl_enrichment/hyper_mtyl_hl60_delta_te_GENESET/hl60_delta_te.txt.summary.pdf > mtyl_enrichment/hyper_mtyl_hl60_delta_te_GENESET/hl60_delta_te.txt.summary.png
do

5. Write README.md draft
    - Write HTML codes which link all plots into a `README.md` format to prepare GitHub friendly report